In [369]:
from selenium import webdriver
from datetime import datetime
import pandas as pd
import numpy as np
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException


# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")


# URL of the webpage containing the dropdown menu
url = f"https://www.basketball-reference.com/leagues/NBA_2024_per_game.html"
driver.get(url)

# Player Ratings

In [370]:
# Have the table only show accurate data
hide_partial_row_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="per_game_stats_toggle_partial_table"]'))
) 

hide_partial_row_button.click()
print('Hide rows with partial data')

Hide rows with partial data


In [371]:
# Get table with data
table = driver.find_element(By.XPATH,'//*[@id="per_game_stats"]')

# Extract table header
headers = table.find_element(By.XPATH, '//*[@id="per_game_stats"]/thead').text.split(' ')

# Extract data from table
tbody = table.find_element(By.XPATH, '//*[@id="per_game_stats"]/tbody')

# Replace rows that have a space with '0'
for data in tbody.find_elements(By.TAG_NAME, 'td'):
    if data.text == '':
        driver.execute_script("arguments[0].innerText = '0';", data)
        
# Get table data
player_rows = [row.text.split(' ') for row in tbody.find_elements(By.CLASS_NAME, 'full_table')]

In [ ]:
player_rows = [row.text.split(' ') for row in tbody.find_elements(By.TAG_NAME, 'td')]
player_rows

In [355]:
player_data = []

for row in player_rows:   
    # Find rows where the player name has two parts 'Kobe Brown'
    if len(row) < 32:
        row.pop(1)
        row.pop(1)
    # Find rows where the player name has three parts 'Greg Brown III'
    elif len(row) == 32:
        row.pop(1)
        row.pop(1)
        row.pop(1)
    player_data.append(row)
    
player_data

[['1',
  'C-PF',
  '24',
  'TOT',
  '31',
  '0',
  '16.5',
  '2.8',
  '6.1',
  '.458',
  '0.5',
  '1.7',
  '.269',
  '2.4',
  '4.5',
  '.529',
  '.495',
  '0.8',
  '1.4',
  '.568',
  '2.0',
  '3.2',
  '5.2',
  '1.5',
  '0.6',
  '0.5',
  '1.1',
  '1.6',
  '6.9'],
 ['2',
  'C',
  '26',
  'MIA',
  '28',
  '28',
  '34.5',
  '8.0',
  '15.7',
  '.511',
  '0.0',
  '0.3',
  '.143',
  '8.0',
  '15.5',
  '.517',
  '.513',
  '5.9',
  '7.6',
  '.783',
  '2.3',
  '8.1',
  '10.4',
  '4.1',
  '1.1',
  '1.0',
  '2.6',
  '2.5',
  '22.0'],
 ['3',
  'SG',
  '23',
  'UTA',
  '40',
  '10',
  '20.4',
  '2.4',
  '5.4',
  '.442',
  '1.1',
  '3.2',
  '.336',
  '1.3',
  '2.2',
  '.598',
  '.542',
  '0.3',
  '0.4',
  '.733',
  '0.8',
  '1.8',
  '2.6',
  '1.0',
  '0.6',
  '0.5',
  '0.7',
  '1.5',
  '6.1'],
 ['4',
  'PF',
  '23',
  'MEM',
  '29',
  '5',
  '23.2',
  '3.9',
  '9.0',
  '.438',
  '1.6',
  '4.6',
  '.341',
  '2.4',
  '4.4',
  '.539',
  '.525',
  '1.0',
  '1.6',
  '.609',
  '1.3',
  '4.2',
  '5.5',
  '1

In [356]:
player_names = []

# Extract player names from table
names = tbody.find_elements(By.CLASS_NAME, 'left')

# Extract names and add to array
for name in names:
    if len(name.text) > 3:
        player_names.append(name.text)

# Close the browser
# driver.close()

player_names

['Precious Achiuwa',
 'Bam Adebayo',
 'Ochai Agbaji',
 'Santi Aldama',
 'Nickeil Alexander-Walker',
 'Grayson Allen',
 'Jarrett Allen',
 'Jose Alvarado',
 'Kyle Anderson',
 'Giannis Antetokounmpo',
 'Thanasis Antetokounmpo',
 'Cole Anthony',
 'OG Anunoby',
 'Ryan Arcidiacono',
 'Deni Avdija',
 'Deandre Ayton',
 'Udoka Azubuike',
 'Ibou Badji',
 'Marvin Bagley III',
 'Amari Bailey',
 'Patrick Baldwin Jr.',
 'LaMelo Ball',
 'Mo Bamba',
 'Paolo Banchero',
 'Desmond Bane',
 'Dalano Banton',
 'Dominick Barlow',
 'Harrison Barnes',
 'Scottie Barnes',
 'RJ Barrett',
 'Charles Bassey',
 'Emoni Bates',
 'Keita Bates-Diop',
 'Nicolas Batum',
 'Bradley Beal',
 'Malik Beasley',
 'MarJon Beauchamp',
 'Jules Bernard',
 'Dāvis Bertāns',
 'Patrick Beverley',
 'Saddiq Bey',
 'Goga Bitadze',
 'Onuralp Bitim',
 'Bismack Biyombo',
 'Anthony Black',
 'Leaky Black',
 'Bogdan Bogdanović',
 'Bojan Bogdanović',
 'Bol Bol',
 'Marques Bolden',
 'Devin Booker',
 'Brandon Boston Jr.',
 'Chris Boucher',
 'James Bou

In [357]:
# Add respective team names to corresponding table data rows
for i in range(len(player_data)):
    player_data[i].insert(1, player_names[i])
    
player_data[179]

['180',
 'Ron Harper Jr.',
 'Jr.',
 'PF',
 '23',
 'TOR',
 '1',
 '0',
 '4.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '0.0',
 '1.0',
 '0.0',
 '0.0',
 '0.0',
 '2.0',
 '0.0']

In [331]:
# Create a pandas Data Frame
player_df = pd.DataFrame(data=player_data, columns=headers)
player_df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C-PF,24,TOT,31,0,16.5,2.8,6.1,...,.568,2.0,3.2,5.2,1.5,0.6,0.5,1.1,1.6,6.9
1,2,Bam Adebayo,C,26,MIA,28,28,34.5,8.0,15.7,...,.783,2.3,8.1,10.4,4.1,1.1,1.0,2.6,2.5,22.0
2,3,Ochai Agbaji,SG,23,UTA,40,10,20.4,2.4,5.4,...,.733,0.8,1.8,2.6,1.0,0.6,0.5,0.7,1.5,6.1
3,4,Santi Aldama,PF,23,MEM,29,5,23.2,3.9,9.0,...,.609,1.3,4.2,5.5,1.7,0.7,0.7,1.1,1.6,10.4
4,5,Nickeil Alexander-Walker,SG,25,MIN,38,12,22.3,2.3,5.6,...,.696,0.4,1.3,1.8,2.4,0.9,0.6,1.0,1.8,6.4


In [332]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rk      522 non-null    object
 1   Player  522 non-null    object
 2   Pos     522 non-null    object
 3   Age     522 non-null    object
 4   Tm      522 non-null    object
 5   G       522 non-null    object
 6   GS      522 non-null    object
 7   MP      522 non-null    object
 8   FG      522 non-null    object
 9   FGA     522 non-null    object
 10  FG%     522 non-null    object
 11  3P      522 non-null    object
 12  3PA     522 non-null    object
 13  3P%     522 non-null    object
 14  2P      522 non-null    object
 15  2PA     522 non-null    object
 16  2P%     522 non-null    object
 17  eFG%    522 non-null    object
 18  FT      522 non-null    object
 19  FTA     522 non-null    object
 20  FT%     522 non-null    object
 21  ORB     522 non-null    object
 22  DRB     522 non-null    ob

In [333]:
# Convert columns with data type object to float
player_df = player_df.apply(pd.to_numeric, errors='ignore', downcast='float')
player_df.dtypes

Rk        float32
Player     object
Pos        object
Age        object
Tm         object
G          object
GS        float32
MP        float32
FG        float32
FGA       float32
FG%       float32
3P        float32
3PA       float32
3P%       float32
2P        float32
2PA       float32
2P%       float32
eFG%      float32
FT        float32
FTA       float32
FT%       float32
ORB       float32
DRB       float32
TRB       float32
AST       float32
STL       float32
BLK       float32
TOV       float32
PF        float32
PTS       float32
dtype: object

In [334]:
player_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      522 non-null    float32
 1   Player  522 non-null    object 
 2   Pos     522 non-null    object 
 3   Age     522 non-null    object 
 4   Tm      522 non-null    object 
 5   G       522 non-null    object 
 6   GS      522 non-null    float32
 7   MP      522 non-null    float32
 8   FG      522 non-null    float32
 9   FGA     522 non-null    float32
 10  FG%     522 non-null    float32
 11  3P      522 non-null    float32
 12  3PA     522 non-null    float32
 13  3P%     522 non-null    float32
 14  2P      522 non-null    float32
 15  2PA     522 non-null    float32
 16  2P%     522 non-null    float32
 17  eFG%    522 non-null    float32
 18  FT      522 non-null    float32
 19  FTA     522 non-null    float32
 20  FT%     522 non-null    float32
 21  ORB     522 non-null    float32
 22  DR

In [335]:
# player_df.dropna()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Precious Achiuwa,C-PF,24,TOT,31,0.0,16.500000,2.8,6.1,...,0.568,2.0,3.2,5.2,1.5,0.6,0.5,1.1,1.6,6.9
1,2.0,Bam Adebayo,C,26,MIA,28,28.0,34.500000,8.0,15.7,...,0.783,2.3,8.1,10.4,4.1,1.1,1.0,2.6,2.5,22.0
2,3.0,Ochai Agbaji,SG,23,UTA,40,10.0,20.400000,2.4,5.4,...,0.733,0.8,1.8,2.6,1.0,0.6,0.5,0.7,1.5,6.1
3,4.0,Santi Aldama,PF,23,MEM,29,5.0,23.200001,3.9,9.0,...,0.609,1.3,4.2,5.5,1.7,0.7,0.7,1.1,1.6,10.4
4,5.0,Nickeil Alexander-Walker,SG,25,MIN,38,12.0,22.299999,2.3,5.6,...,0.696,0.4,1.3,1.8,2.4,0.9,0.6,1.0,1.8,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516,517.0,Delon Wright,PG,31,WAS,16,0.0,17.000000,1.5,4.1,...,0.783,0.4,1.9,2.3,3.3,1.4,0.2,0.6,0.8,4.6
517,518.0,Thaddeus Young,PF,35,TOR,11,3.0,9.000000,1.5,2.5,...,0.333,0.5,1.1,1.6,1.5,0.6,0.1,0.6,1.2,3.1
518,519.0,Trae Young,PG,25,ATL,35,35.0,36.500000,8.5,19.9,...,0.857,0.5,2.5,3.0,10.9,1.5,0.2,4.3,1.9,27.4
519,520.0,Omer Yurtseven,C,25,UTA,23,6.0,11.000000,1.7,3.3,...,0.600,1.5,2.9,4.4,0.6,0.2,0.4,1.0,1.4,3.7


In [336]:
player_df.to_csv('player_ratings.csv', sep=',', encoding='utf-8', index=False)

# Team Ratings

In [337]:
from selenium import webdriver
from datetime import datetime
import pandas as pd
import numpy as np
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")

# URL of the webpage containing the dropdown menu
url = f"https://www.basketball-reference.com/leagues/NBA_2024_ratings.html"
driver.get(url)

In [338]:
# Extract table
table = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="ratings"]'))
)

# Get headers used in table
thead = table.find_element(By.TAG_NAME, 'thead')
headers = [header.text.split() for header in thead.find_elements(By.TAG_NAME,'tr')][1]

# Get data from table
tbody = table.find_element(By.XPATH, '//*[@id="ratings"]/tbody')
team_rows = [row.text.split() for row in tbody.find_elements(By.TAG_NAME, 'tr')]

In [339]:
team_data = []

for row in team_rows:
    # Find rows where the team name has two parts 'Boston Celtics'
    if len(row) == 16:
        row.pop(1)
        row.pop(1)
    # Find rows where the team name has three parts 'Oklahoma City Thunder'
    elif len(row) == 17:
        row.pop(1)
        row.pop(1)
        row.pop(1)
    elif row[0] == 'Unadjusted' or row[0] == 'Rk':
        continue  # Skip headers and other irrelevant rows
    team_data.append(row)
    
team_data

[['1',
  'E',
  'A',
  '29',
  '9',
  '.763',
  '9.21',
  '122.45',
  '113.05',
  '9.41',
  '9.92',
  '122.63',
  '112.51',
  '10.11'],
 ['2',
  'W',
  'NW',
  '26',
  '11',
  '.703',
  '9.05',
  '122.60',
  '113.54',
  '9.06',
  '9.06',
  '122.70',
  '113.63',
  '9.07'],
 ['3',
  'E',
  'A',
  '24',
  '13',
  '.649',
  '8.11',
  '120.56',
  '112.39',
  '8.17',
  '7.29',
  '119.87',
  '112.52',
  '7.35'],
 ['4',
  'W',
  'NW',
  '27',
  '11',
  '.711',
  '5.84',
  '115.69',
  '109.75',
  '5.94',
  '6.84',
  '116.06',
  '109.12',
  '6.94'],
 ['5',
  'W',
  'NW',
  '27',
  '13',
  '.675',
  '5.13',
  '120.22',
  '114.89',
  '5.33',
  '5.15',
  '120.50',
  '115.17',
  '5.33'],
 ['6',
  'W',
  'P',
  '25',
  '13',
  '.658',
  '5.74',
  '121.08',
  '115.28',
  '5.80',
  '5.25',
  '120.91',
  '115.60',
  '5.31'],
 ['7',
  'W',
  'SW',
  '23',
  '16',
  '.590',
  '4.56',
  '118.26',
  '113.59',
  '4.68',
  '4.72',
  '118.77',
  '113.95',
  '4.82'],
 ['8',
  'E',
  'C',
  '26',
  '12',
  '.684

In [340]:
team_names = []

# Extract team names from table
names = tbody.find_elements(By.CLASS_NAME, 'left')

# Extract team names and add to array
for name in names:
    if name.text == 'Team':
        continue
    else:
        team_names.append(name.text)
# Close the browser
driver.close()

team_names

['Boston Celtics',
 'Oklahoma City Thunder',
 'Philadelphia 76ers',
 'Minnesota Timberwolves',
 'Denver Nuggets',
 'Los Angeles Clippers',
 'New Orleans Pelicans',
 'Milwaukee Bucks',
 'New York Knicks',
 'Indiana Pacers',
 'Houston Rockets',
 'Cleveland Cavaliers',
 'Orlando Magic',
 'Dallas Mavericks',
 'Golden State Warriors',
 'Miami Heat',
 'Sacramento Kings',
 'Phoenix Suns',
 'Los Angeles Lakers',
 'Atlanta Hawks',
 'Toronto Raptors',
 'Chicago Bulls',
 'Brooklyn Nets',
 'Utah Jazz',
 'Memphis Grizzlies',
 'San Antonio Spurs',
 'Portland Trail Blazers',
 'Washington Wizards',
 'Detroit Pistons',
 'Charlotte Hornets']

In [341]:
# Add respective team names to corresponding table data rows
for i in range(len(team_data)):
    team_data[i].insert(1, team_names[i])

In [342]:
team_data

[['1',
  'Boston Celtics',
  'E',
  'A',
  '29',
  '9',
  '.763',
  '9.21',
  '122.45',
  '113.05',
  '9.41',
  '9.92',
  '122.63',
  '112.51',
  '10.11'],
 ['2',
  'Oklahoma City Thunder',
  'W',
  'NW',
  '26',
  '11',
  '.703',
  '9.05',
  '122.60',
  '113.54',
  '9.06',
  '9.06',
  '122.70',
  '113.63',
  '9.07'],
 ['3',
  'Philadelphia 76ers',
  'E',
  'A',
  '24',
  '13',
  '.649',
  '8.11',
  '120.56',
  '112.39',
  '8.17',
  '7.29',
  '119.87',
  '112.52',
  '7.35'],
 ['4',
  'Minnesota Timberwolves',
  'W',
  'NW',
  '27',
  '11',
  '.711',
  '5.84',
  '115.69',
  '109.75',
  '5.94',
  '6.84',
  '116.06',
  '109.12',
  '6.94'],
 ['5',
  'Denver Nuggets',
  'W',
  'NW',
  '27',
  '13',
  '.675',
  '5.13',
  '120.22',
  '114.89',
  '5.33',
  '5.15',
  '120.50',
  '115.17',
  '5.33'],
 ['6',
  'Los Angeles Clippers',
  'W',
  'P',
  '25',
  '13',
  '.658',
  '5.74',
  '121.08',
  '115.28',
  '5.80',
  '5.25',
  '120.91',
  '115.60',
  '5.31'],
 ['7',
  'New Orleans Pelicans',
  '

In [343]:
team_ratings = pd.DataFrame(data=team_data, columns=headers)
team_ratings

,Rk,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,1,Boston Celtics,E,A,29,9,.763,9.21,122.45,113.05,9.41,9.92,122.63,112.51,10.11
1,2,Oklahoma City Thunder,W,NW,26,11,.703,9.05,122.60,113.54,9.06,9.06,122.70,113.63,9.07
2,3,Philadelphia 76ers,E,A,24,13,.649,8.11,120.56,112.39,8.17,7.29,119.87,112.52,7.35
3,4,Minnesota Timberwolves,W,NW,27,11,.711,5.84,115.69,109.75,5.94,6.84,116.06,109.12,6.94
4,5,Denver Nuggets,W,NW,27,13,.675,5.13,120.22,114.89,5.33,5.15,120.50,115.17,5.33
5,6,Los Angeles Clippers,W,P,25,13,.658,5.74,121.08,115.28,5.80,5.25,120.91,115.60,5.31
6,7,New Orleans Pelicans,W,SW,23,16,.590,4.56,118.26,113.59,4.68,4.72,118.77,113.95,4.82
7,8,Milwaukee Bucks,E,C,26,12,.684,4.97,122.19,117.28,4.90,4.10,121.34,117.29,4.05
8,9,New York Knicks,E,A,22,16,.579,3.79,119.53,115.50,4.02,3.81,119.14,115.10,4.04
9,10,Indiana Pacers,E,C,23,15,.605,3.18,123.68,120.51,3.17,2.86,123.59,120.72,2.87


In [344]:
team_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rk      30 non-null     object
 1   Team    30 non-null     object
 2   Conf    30 non-null     object
 3   Div     30 non-null     object
 4   W       30 non-null     object
 5   L       30 non-null     object
 6   W/L%    30 non-null     object
 7   MOV     30 non-null     object
 8   ORtg    30 non-null     object
 9   DRtg    30 non-null     object
 10  NRtg    30 non-null     object
 11  MOV/A   30 non-null     object
 12  ORtg/A  30 non-null     object
 13  DRtg/A  30 non-null     object
 14  NRtg/A  30 non-null     object
dtypes: object(15)
memory usage: 3.6+ KB


In [345]:
# Convert selected columns from object to float
team_ratings = team_ratings.apply(pd.to_numeric, errors='ignore', downcast='float')
team_ratings.dtypes

Rk        float32
Team       object
Conf       object
Div        object
W         float32
L         float32
W/L%      float32
MOV       float32
ORtg      float32
DRtg      float32
NRtg      float32
MOV/A     float32
ORtg/A    float32
DRtg/A    float32
NRtg/A    float32
dtype: object

In [346]:
team_ratings.to_csv('team_ratings.csv', sep=',', encoding='utf-8', index=False)